In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import warnings
warnings.filterwarnings("ignore")

In [ ]:
spark = SparkSession.builder \
    .appName("Fact_Table Gold") \
    .enableHiveSupport() \
    .getOrCreate()

25/04/07 20:40:20 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
spark.sql("CREATE DATABASE IF NOT EXISTS gold")

25/04/07 20:40:25 WARN HiveConf: HiveConf of name hive.metastore.db.type does not exist


DataFrame[]

In [4]:
spark.sql("USE gold")

DataFrame[]

In [5]:
hdfs_silver_path ="hdfs:///data/silver/Fact_Table_parquet"
fact_df = spark.read.parquet(hdfs_silver_path)

In [6]:
fact_df.printSchema()

root
 |-- Edition ID: integer (nullable = true)
 |-- Country NOC: string (nullable = true)
 |-- Result ID: integer (nullable = true)
 |-- Athlete ID: integer (nullable = true)
 |-- Position: string (nullable = true)
 |-- Medal Athlete: string (nullable = true)
 |-- Gold Medals: integer (nullable = true)
 |-- Silver Medals: integer (nullable = true)
 |-- Bronze Medals: integer (nullable = true)
 |-- Total Medals: integer (nullable = true)



In [7]:
date_silver_path ="hdfs:///data/silver/Date_parquet"
date_df = spark.read.parquet(date_silver_path)

In [8]:
fact_with_date_id = fact_df.join(
    date_df.select("Date_id"),
    fact_df["Edition ID"] == date_df["Date_id"],
    how="left"
)

In [9]:
fact_with_date_id.printSchema()

root
 |-- Edition ID: integer (nullable = true)
 |-- Country NOC: string (nullable = true)
 |-- Result ID: integer (nullable = true)
 |-- Athlete ID: integer (nullable = true)
 |-- Position: string (nullable = true)
 |-- Medal Athlete: string (nullable = true)
 |-- Gold Medals: integer (nullable = true)
 |-- Silver Medals: integer (nullable = true)
 |-- Bronze Medals: integer (nullable = true)
 |-- Total Medals: integer (nullable = true)
 |-- Date_id: integer (nullable = true)



In [10]:
fact_with_date_id.write.format("hive").mode("overwrite").saveAsTable("gold.Fact_Table")

25/04/07 20:28:28 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


In [ ]:
fact_with_date_id.write.mode("overwrite").parquet("hdfs:///data/gold/Fact_Table_parquet")